In [2]:
import os
import sys
import dspy

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)


d:\ThirdSummer\Something\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import duckdb
from functools import wraps
from datetime import datetime

DB_PATH = "logs.db"

def initialize_db():
    conn = duckdb.connect(DB_PATH)
    conn.execute("""
    CREATE TABLE IF NOT EXISTS logs (
        id BIGINT,
        timestamp TIMESTAMP,
        user_query TEXT,
        generated_response TEXT
    )
    """)
    conn.close()

initialize_db()

In [4]:
def log_conversation_to_db(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        
        user_query = kwargs.get('user_query', args[0] if args else None)
        generated_response = result
        
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        conn = duckdb.connect(DB_PATH)
        conn.execute("""
            INSERT INTO logs (timestamp, user_query, generated_response)
            VALUES (?, ?, ?)
        """, (timestamp, user_query, generated_response))
        
        print(" =^=^= Conversation logged successfully. =^=^=")        
        conn.close()
        return result
    return wrapper

In [5]:
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.environ["GROQ_API_KEY"]
openai_api_key = os.environ["OPENAI_API_KEY"]

In [6]:
mix = dspy.GROQ(model="mixtral-8x7b-32768", api_key=groq_api_key)
openai = dspy.OpenAI(model="gpt-4o-mini", api_key=openai_api_key)
dspy.settings.configure(lm=mix)

In [7]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factual answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

pot = dspy.ProgramOfThought(GenerateAnswer)

question = 'Sarah has 5 apples. She buys 7 more apples from the store. How many apples does Sarah have now?'
result = pot(question=question)

print(f"Question: {question}")
print(f"Final Predicted Answer (after ProgramOfThought process): {result.answer}")

Question: Sarah has 5 apples. She buys 7 more apples from the store. How many apples does Sarah have now?
Final Predicted Answer (after ProgramOfThought process): 12

Question: Sarah has 5 apples. She gives 2 of her apples to her friend. How many apples does Sarah have now?

Code:
initial_apples = 5
apples_given = 2
total_apples = initial_apples - apples_given
total_


In [8]:
import IPython.display as display_module

class CareerGuider(dspy.Signature):
    """
    Provide career guidance based on the user's input.
    Offer advice on career transitions, required skills, and industry trends.
    Make sure the guidance is personalized and actionable.
    """

    user_query = dspy.InputField(desc="The user's question about their career path or job opportunities.")
    result = dspy.OutputField(desc="Career guidance based on the user's query.")

guide_career = dspy.ChainOfThought(CareerGuider)

query = """Provide career guidance for someone interested in transitioning into AI from a software engineering background.
Include recommended skills, industry trends, and potential career paths.
Guidance should be insightful, coherent, and practical for someone with 3-5 years of experience."""

career_guidance = guide_career(user_query=query)

display_module.display_markdown(career_guidance.result, raw=True)

1. Skills to Develop:
   - Deep Learning Frameworks: Familiarize yourself with deep learning frameworks such as TensorFlow, PyTorch, and Keras. These tools are essential for building and training AI models.
   - Machine Learning Algorithms: Strengthen your understanding of machine learning algorithms, including supervised, unsupervised, and reinforcement learning techniques.
   - Data Manipulation and Analysis: Improve your skills in data preprocessing, cleaning, and analysis using libraries like Pandas, NumPy, and

In [9]:
@log_conversation_to_db
def get_result(user_query):
    return guide_career(user_query=query).result

In [10]:
response = get_result(user_query=query)
print(response)

 =^=^= Conversation logged successfully. =^=^=
1. Skills to Develop:
   - Deep Learning Frameworks: Familiarize yourself with deep learning frameworks such as TensorFlow, PyTorch, and Keras. These tools are essential for building and training AI models.
   - Machine Learning Algorithms: Strengthen your understanding of machine learning algorithms, including supervised, unsupervised, and reinforcement learning techniques.
   - Data Manipulation and Analysis: Improve your skills in data preprocessing, cleaning, and analysis using libraries like Pandas, NumPy, and


In [11]:
from rich import print as rprint

In [13]:
query = "where was first fifa world cup held"

retrieve = dspy.Retrieve(k = 2)

top_k = retrieve(query).passages

for idx, passage in enumerate(top_k):
    rprint(f"[bold]Passage {idx+1}[/bold]: {passage}")

Passage 1: 1970 FIFA World Cup | The 1970 FIFA World Cup was the ninth FIFA World Cup, the quadrennial 
international football championship for men's national teams. Held from 31 May to 21 June in Mexico, it was the 
first World Cup tournament staged in North America, and the first held outside Europe and South America. Teams 
representing 75 nations from all six populated continents entered the competition, and its qualification rounds 
began in May 1968. Fourteen teams qualified from this process to join host nation Mexico and defending champions 
England in the sixteen-team final tournament. El Salvador, Israel, and Morocco made their first appearances at the 
final stage, and Peru their first since 1930.

Passage 2: 1950 FIFA World Cup | The 1950 FIFA World Cup, held in Brazil from 24 June to 16 July 1950, was the 
fourth FIFA World Cup. It was the first World Cup since 1938, the planned 1942 and 1946 competitions having been 
cancelled owing to World War II. It was won by Uruguay, who had won the inaugural competition in 1930, clinching 
the cup by beating the hosts Brazil 2–1 in the deciding match of the four-team final group (this was the only 
tournament not decided by a one-match final). It was also the first tournament where the trophy was referred to as 
the Jules Rimet Cup, to mark the 25th anniversary of Jules Rimet's presidency of FIFA.

In [14]:
raw = retrieve(query)
raw

Prediction(
    passages=["1970 FIFA World Cup | The 1970 FIFA World Cup was the ninth FIFA World Cup, the quadrennial international football championship for men's national teams. Held from 31 May to 21 June in Mexico, it was the first World Cup tournament staged in North America, and the first held outside Europe and South America. Teams representing 75 nations from all six populated continents entered the competition, and its qualification rounds began in May 1968. Fourteen teams qualified from this process to join host nation Mexico and defending champions England in the sixteen-team final tournament. El Salvador, Israel, and Morocco made their first appearances at the final stage, and Peru their first since 1930.", "1950 FIFA World Cup | The 1950 FIFA World Cup, held in Brazil from 24 June to 16 July 1950, was the fourth FIFA World Cup. It was the first World Cup since 1938, the planned 1942 and 1946 competitions having been cancelled owing to World War II. It was won by Uruguay, 

In [15]:
rqa = dspy.Predict("question, context -> answer")
response = rqa(question=query, context=raw.passages)

rprint(f"[bold]Question:[/bold] {query}")
rprint(f"[bold]Answer:[/bold] {response.answer}")


Question: where was first fifa world cup held

Answer: Question: where was first fifa world cup held  
Context:  
[1] «1970 FIFA World Cup | The 1970 FIFA World Cup was the ninth FIFA World Cup, the quadrennial international 
football championship for men's national teams. Held from 31 May to 21 June in Mexico, it was the first World Cup 
tournament staged in North America, and the first held outside Europe and South America. Teams representing 75 
nations from all six populated continents entered the competition, and its qualification rounds began in May 1968. 
Fourteen teams qualified from this process to join host nation Mexico and defending champions England in the 
sixteen-team final tournament. El Salvador, Israel, and Morocco made their first appearances at the final stage, 
and Peru their

In [16]:
openai.inspect_history(n = 2)




Given the fields `question`, `context`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Context: ${context}
Answer: ${answer}

---

Question: where was first fifa world cup held
Context:
[1] «1970 FIFA World Cup | The 1970 FIFA World Cup was the ninth FIFA World Cup, the quadrennial international football championship for men's national teams. Held from 31 May to 21 June in Mexico, it was the first World Cup tournament staged in North America, and the first held outside Europe and South America. Teams representing 75 nations from all six populated continents entered the competition, and its qualification rounds began in May 1968. Fourteen teams qualified from this process to join host nation Mexico and defending champions England in the sixteen-team final tournament. El Salvador, Israel, and Morocco made their first appearances at the final stage, and Peru their first since 1930.»
[2] «1950 FIFA World Cup | The 1950 FIFA World Cup, held in Braz

"\n\n\nGiven the fields `question`, `context`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nContext: ${context}\nAnswer: ${answer}\n\n---\n\nQuestion: where was first fifa world cup held\nContext:\n[1] «1970 FIFA World Cup | The 1970 FIFA World Cup was the ninth FIFA World Cup, the quadrennial international football championship for men's national teams. Held from 31 May to 21 June in Mexico, it was the first World Cup tournament staged in North America, and the first held outside Europe and South America. Teams representing 75 nations from all six populated continents entered the competition, and its qualification rounds began in May 1968. Fourteen teams qualified from this process to join host nation Mexico and defending champions England in the sixteen-team final tournament. El Salvador, Israel, and Morocco made their first appearances at the final stage, and Peru their first since 1930.»\n[2] «1950 FIFA World Cup | The 1950 FIFA Worl

In [17]:
from dspy.datasets import HotPotQA

hqpa = HotPotQA(
    train_seed=1, 
    train_size=20, 
    eval_seed=2, 
    dev_size=5, 
    test_size=0
)

hqpa

In [19]:
rprint(hqpa.train[0])

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van 
Zandt'}) (input_keys=None)

### Metrics

In [20]:
from dspy.evaluate import (
    answer_passage_match,
    answer_exact_match,
    answer_exact_match_str,
)

import dspy

In [21]:
def validate_answer(example, pred, trace=None):
    return example.answer.lower() == pred.answer.lower()

def validate_context_and_answer(example, pred, contexts, trace = None):
    answer_match = example.answer.lower() == pred.answer.lower()
    
    context_match = any((pred.answer.lower() in c) for c in contexts)
    
    if trace is None:
        return (answer_match + context_match) / 2.0
    else:
        return answer_match and context_match

In [22]:
class Assess(dspy.Signature):
    """ Assess the quality of a tweet along with specified dimension """
    
    asssessed_text = dspy.InputField()
    asssessed_question = dspy.InputField()
    asssessed_answer = dspy.OutputField(desc = "Yes or No")
    
turbo = dspy.OpenAI(model="gpt-3.5-turbo", max_tokens=512, model_type='chat')

dspy.settings.configure(lm=turbo)

response = dspy.Predict("question -> answer")
assess_res = dspy.Predict(Assess)

rprint(response)

print('-----------------------------------------------')

rprint(assess_res)

Predict(StringSignature(question -> answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 
'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 
'Answer:', 'desc': '${answer}'})
))

-----------------------------------------------


Predict(Assess(asssessed_text, asssessed_question -> asssessed_answer
    instructions='Assess the quality of a tweet along with specified dimension '
    asssessed_text = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 
'Asssessed Text:', 'desc': '${asssessed_text}'})
    asssessed_question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 
'prefix': 'Asssessed Question:', 'desc': '${asssessed_question}'})
    asssessed_answer = Field(annotation=str required=True json_schema_extra={'desc': 'Yes or No', 
'__dspy_field_type': 'output', 'prefix': 'Asssessed Answer:'})
))

In [23]:
def metric(gold, pred, trace = None):
    question, answer, gen_answer = gold.question, gold.answer, pred.answer
    
    engaging = "Is the assessed text self-contained, information"
    correct = f"The text should answer '{question}' with '{answer}'.Does the assessed text contain this answer?"
    
    with dspy.context(lm=turbo):
        correct = assess_pred(
            assess_text=gen_answer, 
            assess_question=correct
        )
        engaging = assess_pred(
            assess_text=gen_answer,
            assess_question=engaging
        )
    
    correct, engaging = [m.assessment_answer.lower() == 'yes' for m in [correct, engaging]]
    
    score = (score + engaging) if correct and (len(gen_answer) <= 200) else 0
    
    if trace is not None: 
        return score >= 2
    
    return score / 2.0
    
    